# DeepNN for Sink Particles
> Created Oct. 2024 <br>
> Nikhil Bisht<br>

In [5]:
# standard system modules
import os, sys

# standard module for tabular data
import pandas as pd

# standard module for array manipulation
import numpy as np

# standard statistical module
import scipy.stats as st

# standard module for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt
mp.rcParams.update(mp.rcParamsDefault)
%matplotlib inline

# standard research-level machine learning toolkit from Meta (FKA: FaceBook)
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import xgboost as xgb


# set a seed to ensure reproducibility
seed = 128
rnd  = np.random.RandomState(seed)

## Constants

In [6]:
DATAFILE  = '/data/cb1/nbisht/anvil_scratch/projects/128/B2/datasets/nb101_ML_dataset.csv'
MODELFILE = 'nnmodel.dict'

NTRAIN = 1600000
NVALID =  100000
NTEST  =  300000 #roughly

TARGET = ['O_Clump_X', 'O_Clump_Y',	'O_Clump_Z', 'O_Clump_Vx', 'O_Clump_Vy', 'O_Clump_Vz', 'O_Clump_density','O_t_end']
FEATURES = ['Clump_id', 'X', 'Y', 'Z', 'Vx', 'Vy', 'Vz','Density', 't_hard']

n_input = len(FEATURES)
n_output = len(TARGET)

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cpu')

print(f'Available device: {str(DEVICE):4s}')
print(n_input, n_output)

Available device: cpu 
9 8


## Load data

In [103]:
df = pd.read_csv(DATAFILE)
df['Clump_id'] = df['Clump_id'].astype('category')
print(len(df))
#df = df.sample(frac=1).reset_index(drop=True)
df.head()
#BIN CLUMP_ID -1 to reduce number 
reduce_imbalance = False
if reduce_imbalance:
    every_other = 5 #skip these many indices when filtering dataset. Will reduce the dataset by this many times
    index_d0_1 = df[(df['Clump_id']==-1) & (df['O_Clump_density']<1)].index
    remove_these = set(index_d0_1) - set(index_d0_1[::every_other])
    df = df.drop(list(remove_these))
    index_d1_10 = df[(df['Clump_id']==-1) & (df['O_Clump_density']<10) &(df['O_Clump_density']>1)].index
    remove_these = set(index_d1_10) - set(index_d1_10[::every_other])
    df = df.drop(list(remove_these))
    index_d10_inf = df[(df['Clump_id']==-1) & (df['O_Clump_density']>10)].index
    remove_these = set(index_d10_inf) - set(index_d10_inf[::every_other])
    df = df.drop(list(remove_these))
    print("df length is now:",len(df))

2097152


## Split data

In [159]:
X, y = df[FEATURES], df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# Training

In [173]:
#%%writefile nnmodel.py
def define_and_train_model(n = 1000, cross_validation_nfold = 0):
    try:
        del model
    except:
        print("No model to delete")
    
    # Define hyperparameters
    params = {"objective": "reg:squarederror"}#, "tree_method": "gpu_hist"}
    evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
    model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    evals=evals,
    verbose_eval=10, # Every ten rounds
    early_stopping_rounds=50, #Activate early stopping
    )
    print()
    if cross_validation_nfold>0:
        results = xgb.cv(
        params, dtrain_reg,
        num_boost_round=n,
        nfold=cross_validation_nfold,
        verbose_eval=10, # Every ten rounds
        early_stopping_rounds=50,
        )
        best_rmse = results['test-rmse-mean'].min()
        print(best_rmse)
    return model
model = define_and_train_model()

No model to delete
[0]	train-rmse:4998.35020	validation-rmse:5200.24776
[10]	train-rmse:1027.43763	validation-rmse:1047.92885
[20]	train-rmse:943.93452	validation-rmse:993.52073
[30]	train-rmse:880.53813	validation-rmse:953.84419
[40]	train-rmse:834.90057	validation-rmse:929.72387
[50]	train-rmse:799.74659	validation-rmse:910.55595
[60]	train-rmse:762.26698	validation-rmse:892.69499
[70]	train-rmse:738.67347	validation-rmse:882.89868
[80]	train-rmse:707.68644	validation-rmse:873.12361
[90]	train-rmse:691.73285	validation-rmse:866.86301
[100]	train-rmse:663.78738	validation-rmse:861.64499
[110]	train-rmse:652.79387	validation-rmse:861.58419
[120]	train-rmse:628.42480	validation-rmse:859.07892
[130]	train-rmse:615.01498	validation-rmse:855.88657
[140]	train-rmse:607.93181	validation-rmse:853.42314
[150]	train-rmse:591.74213	validation-rmse:851.82921
[160]	train-rmse:570.84470	validation-rmse:849.24885
[170]	train-rmse:564.51029	validation-rmse:848.50949
[180]	train-rmse:554.20686	validat

In [174]:
import sklearn.metrics as skm
# standard measures of model performance
preds = model.predict(dtest_reg)
preds_df = pd.DataFrame(preds, index=X_test.index)
#Core rmse
core_indices = X_test[X_test["Clump_id"] !=-1].index
rmse = skm.mean_squared_error(y_test.loc[core_indices], preds_df.loc[core_indices])
r2 = skm.r2_score(y_test.loc[core_indices],preds_df.loc[core_indices])
print("For Cores, ",rmse,r2)

#Non_Core rmse
non_core_indices = X_test[X_test["Clump_id"] ==-1].index
rmse = skm.mean_squared_error(y_test.loc[non_core_indices], preds_df.loc[non_core_indices])
r2 = skm.r2_score(y_test.loc[non_core_indices],preds_df.loc[non_core_indices])
print("For Non Cores, ",rmse,r2)

For Cores,  747171.0932774782 0.9963633422429186
For Non Cores,  683128.8908151424 -2051313306989260.2


## Stats
### Max Depth, Default = 6
#### Reg: Mean Squared Error (~4 min to train)
- For Cores, $RMSE: 1019423.8676899462; R2:                   0.9963610195367045$
- For Non Cores, $RMSE: 691423.0340961919; R2: -2051313306989260.2$   
#### Reg: Mean Absolute Error   (~107 min to train)
For Cores,  7746916.249471188 0.9932377146982748
For Non Cores,  1261673.1667665534 -2461632269410218.0

#### Reg: Pseudo Huber Error, delta = 0.001 (~20s min to train)
For Cores,  5251149496.88155 -16153185246.33833
For Non Cores,  1215310709.10951 -3.0784832358542043e+39

### Max Depth=7
#### Reg: Mean Squared Error (~4 min to train)
For Cores,  747171.0932774782 0.9963633422429186
For Non Cores,  683128.8908151424 -2051313306989260.2 